In [ ]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import corenlp as cnlp

In [ ]:
def treat_text(text_, remove=['\n', '\r', '\t', '\l', '\f', '\p']):
    text = text_
    if text is None:
        return ''
    for r in remove:
        text = text.replace(r, '')
    return text.strip()

### Harem

In [ ]:
%%time

DIVIDERS = ['.', ';', '?', '!']
CODE_SENTENCE = 0
words = []
pos = []
graphics = []
sentences_code = []
bio_classes = []
biose_classes = []
classes = []
def graphic_feature(word):
    if word.isupper():
        return 4
    if word.istitle():
        return 3
    if word.isdigit():
        return 2
    if not word.isalpha():
        return 1
    return 0;
    

def extract_features(text_, class_=None):
    #global words, pos, graphics, sentences_code, bio_classes, biose_classes, classes
    global CODE_SENTENCE
    import time
    while True:
        _text = treat_text(text_)
        try:
            words_, pos_ = cnlp.tokens_pos(_text)
            break
        except:
            print("Connection refused by the server.. Sleep for 1 seconds")
            print('=', text_)
            time.sleep(1)
    if len(words_) != len(pos_):
        print('WARNING:', len(words_), '!=', len(pos_), '\n', words_, pos_)
    else:
        for i in range(len(words_)):
            w = words_[i]
            p = pos_[i]
            words.append(w)
            pos.append(p)
            graphics.append(graphic_feature(w))
            sentences_code.append(CODE_SENTENCE)
            classes.append(class_)
            if class_ is not None:
                if i == 0:
                    bio_classes.append('B')
                    if len(words_) == 1:
                        biose_classes.append('S')
                    else:
                        biose_classes.append('B')
                else:
                    bio_classes.append('I')
                    if i+1 == len(words_):
                        biose_classes.append('E')
                    else:
                        biose_classes.append('I')
            else:
                bio_classes.append('O')
                biose_classes.append('O')
            if w.strip() in DIVIDERS:
                CODE_SENTENCE += 1


path_harem = "data/CDSegundoHAREM.xml"

root = ET.parse(path_harem).getroot()
docs = root.findall('DOC')


for i_, doc in enumerate(docs):
    print(round((i_+1)/len(docs)*100,2), '%')
    for j_, p in enumerate(doc.findall('P')):
        extract_features(p.text)
        for t in p:
            if t.tag == 'EM':
                if 'CATEG' in t.attrib:
                    extract_features(t.text, t.attrib['CATEG'].split('|')[0])
                else:
                    extract_features(t.text, 'OUTRO')
            elif t.tag == 'ALT':
                if 'CATEG' in t[0].attrib:
                    extract_features(t[0].text, t[0].attrib['CATEG'].split('|')[0])
                else:
                    extract_features(t[0].text, 'OUTRO')
            elif t.tag == 'OMITIDO':
                text = t.text
                if text is None:
                    text = ''
                for tt in t:
                    if tt.text is not None:
                        text += ' ' + tt.text
                    b_ = False
                    if tt.tag == 'ALT':
                        b_ = True
                        for ttt in tt:
                            text += ' ' + ttt.text
                            if ttt.tail is not None:
                                text += ' ' + ttt.tail
                    if tt.tail is not None:
                        text += ' ' + tt.tail
                    if b_:
                        break
                extract_features(text)
            if t.tail is not None:
                extract_features(t.tail)
        CODE_SENTENCE += 1        
            
    
    #sentences = cnlp.sentences_tokens_pos(doc.text)
    #print(doc.text)
    
    


In [ ]:
df = pd.DataFrame({'Word':words, 'Pos':pos, 'Graphic':graphics, 
                   'sentence_code':sentences_code, 'bio': bio_classes,
                   'biose': biose_classes, 'class': classes})

In [ ]:
df

In [ ]:
df.to_csv('harem.csv', index=False)

### CONLL 2003

In [ ]:
lines

In [27]:
%%time
import os
import pandas as pd


lines = open('data/conll2003_train.txt').readlines()
lines1 = lines[1:]
lines = open('data/conll2003_validation.txt').readlines()
lines1 += lines[1:]
lines = open('data/conll2003_test.txt').readlines()
lines1 += lines[1:]
lines = lines1

CODE_SENTENCE = 0
words = []
pos = []
graphics = []
sentences_code = []
bio_classes = []
biose_classes = []
classes = []
def graphic_feature(word):
    if word.isupper():
        return 4
    if word.istitle():
        return 3
    if word.isdigit():
        return 2
    if not word.isalpha():
        return 1
    return 0;
    

def extract_features(word_, pos_, class_, a_bio):
    #global words, pos, graphics, sentences_code, bio_classes, biose_classes, classes
    global CODE_SENTENCE

    words.append(word_)
    pos.append(pos_)
    graphics.append(graphic_feature(word_))
    sentences_code.append(CODE_SENTENCE)

    
    if class_ != 'O':
        bio = class_.split('-')[0]
        class__ = class_.split('-')[1]
        classes.append(class__)
        bio_classes.append(bio)
        if bio == 'B':
            print(a_bio)
            if a_bio == 'I':
                biose_classes.append('B')
            else:
                biose_classes.append('S')
        else:
            if a_bio == 'I':
                biose_classes.append('I')
            else:
                biose_classes.append('E')
    else:
        classes.append('')
        bio_classes.append('O')
        biose_classes.append('O')


for i, line in enumerate(lines):
    if line == '\n':
        CODE_SENTENCE += 1
    else:
        features = line.split(' ')
        class_ = features[3].replace('\n','')
        a_bio = ''
        if i+1 < len(lines) and lines[i+1] != '\n':
            a_features = lines[i+1].split(' ')
            if a_features[3].replace('\n', '') != 'O':
                a_bio = a_features[3].split('-')[0].replace('\n','')
        extract_features(features[0], features[1], class_, a_bio)
    




I

I



I
I


I
I

I




I




I












I

I
I








I

I





I







I


I



I
I




I

I


I














I
I



I



I
I
I








I
I
I

I
I



I
I






I
I


I
I










I

I
I





I



I



I





I












I



I
I

I
I


I









I







I








I



I








I







I
I

I

I



B


I
I
I



I

I

I



I

I



I

I





I






I









I



I
I



I
I



I
I





I





I

I
I

I






I
I




I











I


I


B


I


I




I


I







I







I

I




B








I

I






I

I



I





I
B























I

I




I
















I




I







I
I



I
I
I
I
I
I

I



I
I
I

I
I

I

I

I

B





I
I





I


I





I


I


I
I


I








I

I
I



I


I


I














I


I
I

I
I
I

I
I
I
I
I
I
I

I

I
I


I

I
I
I
I
I
I
I

I

I
I
I
I
I

I

I
I


I
I
I

I
I

I



I
I
I
I
I
I

I
I

I

I

I
I
I
I
I
I
I
I





I

I
I

I
I
I
I

I

I



I

I
I

I


I

I

I
I
I
I
I
I

I


I
I

I

I

I

I

I

I

I

I

I
I
I

I

I

I

I
I
I

I

I


I

I
I

I
I

I
I
I

I





I














I
I









I
I

I
I


I

I







I
I





I
I
I



I


I

I





I

I
I



I


I


I





I
I
I
I
I
I
I





I

I


I
I
I

I
I





I
I
I
I
I


I
I
I



I
I
I
I



I

I
I
I

I
I
I
I

I


I
I
I
I
I

I


I
I
I
I

I
I
I
I
I










I



I



I


I
I






I









I
I
I




I

I
I
B
I

I

I
I
I



I




I

B
I
B
I





I




I








I
I



I

I






I
I





I


I

I



I






I








I



I

























I






I





I


I
I
I

I
I
I

I
I
I
I

I

I

I

I

I

I

I
I
I

I
I
I
I

I
I
I
I
I

I
I
I
I
I
I
I

I
I
I
I
I
I
I
I

I
I
I
I
I

I
I







I


I



I
I











I
I

I
I
I



I
I


I


I




I
I
I
I
I
I
I
I
I
I
I
I
I
I
I
I


I

I
I
I

I

I
















I
I
I
I
I
I




I




I
I













I

I


I
I


I


I
I




I
I
I

I


I

I
I

I





I
I
I
I
I






I




I


I






I



I
I

I

I









I





I

I
I


I
I
I
I

I

I

I




I



I







I




I

I

I

I

I

I

























I












I






I







I



I


I

I





I






I

I





I
I





I













I

I


I

I

I












I

I









I















I

I





I

I





I






I



I







I

I


I
I




I


I
I
I




I



I

I

I
B
I





I
I


I




I

I
B



I



I


I
I






I




I

I
I
I

I
I
I

I
I

I

B
I
I
I
I





I

I
B


I
I



I

I

I
I


I


I




I







I


I
B
I





I





I
I
I





I




I

I
I
I
I
I
I

I
I
I

I
I
I

I
I

I






I


I
I



I
I
I

I


I
I




I
I

I



I


I
I





I


I




I


I




I
I


I
I







I
I

I
I











I
I



I

I






B


I
I




I

I

I





I



I









I

I




















I

I

I








I















I

I


I



I

I


I


I
I
I










I

I

I


I

I

I


I

I


I
I
I



I




I



I

I

I
I


I


I










I
I

I

I

I
I

I

I

I






I








I








I

I
I
I
I

I






I










B

I






I
I
I
I
I
I
I
I
I
I

I


I


I

I
I






I


I

B

I

I
B

I
B



I


I


I
B








I


I
I

I
I



I
I



I





I





I


I

I






I
I
I
I

I
I
I
I
I


I
I











I




B

I

I


I
I


I
I
I


I
I

I
I


I
I


I



I

I
I
I
I
I
I







I
I

I





I





I




I

I
I






I







I

I


I
I
I


I







I
I
I




I
I


I

I


I
I






I

I



I
B




I

I
I


I




I
I
I




I
I

I
I
I


I
I

I


I
I

I
I
I

I
I
I

I


I


I
I
I






I


I


I

I
I

I

I
I
I
I

I

I
I
I
I

I
I


I




I


I









I



I


I
I
I
B
I
I
I

I
I
I
I

I


I
I

I




I
I



I

I
I



I
I
I
I
I






I
I




I

I


I
I


I

I

I

I


I


I
I



I


I


I
I

I




I


I


I
I
I

I


I
I



I

I


I
I
I


I
I
I
I


I

I

I

I
I


I
I

I
I



I






I
I

I
I




I
I





I
I

I

I

I

I


B


I


I
I
I



I





I

I
I
I
I

I

I











I

I




I
B





I






I









I
I














I
I


I





I


I

I

I
I





I

I

I
I




I

















I
I






I

I


I












I
I

I





I
I






I













I





I



I





I


I



I

I
I

I
I


I

I
I


I








I
I



I







I


I
I












I






I
I
I










I
I




I








I

I




I




I
I
I
I
I

B
I

I
I





I



I

I
I

I























I
I

I



I



I

I
I


I












I



I







I


I









I




I
I
I


I

I


I

I

I

I

I
I

I


I
I
I
I







I
I

I
I

I
I
I

I

I

I






I
I


I




I






I
I
I





I





I













I
I
I


I
I
I





I




I










I
I











I

B

I

I

I















I
I

I
I

I
I



I
I









I

I














I

I



I





I












I



I


















I



I


I


I





I
I

I


I
I


I

I
I
I









I
I



I

I

I
I

I

I



I
I

I

I

I
I

I












I
I

I






I

I
I



I









I

I

I




I

I
I


I
B

I
I


I



I


I

I

I





I

I

I
I
I



I


I
B



I

I
I


I



I


I

I


I






I








I
I



I


I

I
I

I

I
I





I



I

I

I

I
I
I
I
I


I

I

I


I

I

I
I

I

I

I
I
I

I
I

I

I
I
I

I
I

I
I
I

I


I

I

I
I

I
I

I
I








I

I
I
I


I


I


I


I


I


I


I


I


I

I
I


I


I


I



I


I










I











I

I

I











I
















I





I


I
I











I



















I












I

I
I
I

I

I



I
I


I


I
I
I







I


I


I


I


I



I




I






I
I
I

I


I
I




I
B
I
I


I
I


I





I
I






I
B
I




I


I















B
I




I


B




I
I














I


I





I





























I








I





I


B
I
I
I
I

I


I



I




I

I


I
I




I










I

I




I



I


B
I
B

I






I
I
I
I
I
I






I

I
I

I









I
I
I

I






I
I



I


I







I


I
I
I

I
I

I



I
I



I


I
I




I


I

I


I



B
I

I

I





I

I

I

I



I

I








I
I


I
I







I
I
I








I

I



I



I


I
I










I
I

I

I









I











I
I

I



I

I

I

I





I


I



I






I




I
I



I
I


I



I















B


I




I











I





I
I

















I


I
I

I

B
I
I
I


I
I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I

I


I


I

I
I

I

I
I

I
I


I

In [28]:
df = pd.DataFrame({'Word':words, 'Pos':pos, 'Graphic':graphics, 
                   'sentence_code':sentences_code, 'bio': bio_classes,
                   'biose': biose_classes, 'class': classes})

In [30]:
df.to_csv('conll2003.csv', index=False)